In [2]:

# Step 2: Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import joblib
import json
from ibm_watson_machine_learning import APIClient

# Step 3: Load Dataset
df = pd.read_csv('C:/Users/satvi/OneDrive/Desktop/TrafficTelligence/traffic volume.csv')

# Feature Engineering
df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['Time'], format='%d-%m-%Y %H:%M:%S')
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
df.drop(['date', 'Time', 'timestamp'], axis=1, inplace=True)

X = df.drop('traffic_volume', axis=1)
y = df['traffic_volume']

cat_cols = ['holiday', 'weather']
num_cols = ['temp', 'rain', 'snow', 'hour', 'dayofweek']

# Preprocessing + Model Pipeline
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), num_cols)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=50, random_state=42))
])

# Train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Save model locally
joblib.dump(pipeline, 'traffic_volume_model.pkl')


['traffic_volume_model.pkl']

In [3]:
# Step 4: IBM Cloud Credentials
wml_credentials = {
    "apikey": "YOUR_API_KEY",
    "url": "https://us-south.ml.cloud.ibm.com"
}

client = APIClient(wml_credentials)

# Step 5: Set project space ID (Use the correct space from IBM Cloud)
client.set.default_space("YOUR_SPACE_ID")


Error getting IAM Token.
Reason: <Response [400]>


WMLClientError: Error getting IAM Token.
Reason: <Response [400]>

In [ ]:
# Step 6: Store the model to WML
model_meta = {
    client.repository.ModelMetaNames.NAME: "Traffic Volume Prediction Model",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: client.software_specifications.get_id_by_name("default_py3.10")
}

published_model = client.repository.store_model(
    model='traffic_volume_model.pkl',
    meta_props=model_meta,
    training_data=None,
    training_target=None
)

model_id = published_model["metadata"]["id"]
print("✅ Model ID:", model_id)


In [ ]:
# Step 7: Create Deployment
deployment = client.deployments.create(
    artifact_uid=model_id,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "Traffic Volume Deployment",
        client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)

deployment_id = deployment["metadata"]["id"]
scoring_endpoint = client.deployments.get_scoring_href(deployment)
print("✅ Deployment ID:", deployment_id)
print("🚀 Scoring Endpoint:", scoring_endpoint)


In [ ]:
# Step 8: Use deployed model for inference
scoring_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ['holiday', 'weather', 'temp', 'rain', 'snow', 'hour', 'dayofweek'],
        "values": [['None', 'Clear', 295.5, 0.0, 0.0, 14, 2]]
    }]
}

prediction = client.deployments.score(scoring_endpoint, scoring_payload)
print("📊 Prediction:", prediction)
